## Langchain Basic Usage (LLM + ChatModel only)
Langchain的框架下面有主要分為兩種Model
- LLM
- Chat

由於Langchain並不提供任何Model，所以LLM相關的library都不在langchain-core裡面

#### Langchain最基本的用法
`Prompt -> LLM -> Response`

##### Loading environment variables

In [1]:
%load_ext dotenv
%dotenv

import os

##### Basic Prompts

In [2]:
# Everything starts with LLM
from langchain_core.messages import HumanMessage

msg = HumanMessage(content='茶餐廳是什麼？')

##### 使用ChatModel (e.g. TogetherAI)

In [3]:
from langchain_together import ChatTogether
model = ChatTogether(
    together_api_key=os.environ['KEY_TOGETHERAI'],
    model="Qwen/Qwen1.5-72B-Chat",
)

In [ ]:
res = model.invoke([msg])

##### 利用output_parser將output變得更易讀

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser(res)